## Reference
- Chris McCormick 
- BERT Word Embeddings Tutorial<br>
http://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/<br>
- colab<br>
https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX#scrollTo=0NmMdkZO8R6q<br>
- Youtube<br>
https://www.youtube.com/watch?v=Hnvb9b7a_Ps&feature=youtu.be<br>
- BERT Fine-Tuning Tutorial with PyTorch<br>
https://mccormickml.com/2019/07/22/BERT-fine-tuning/<br>
- Naver<br>
https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=P58qy4--s5_x<br>


#1.Set-up

##1.1. Using Colab GPU for Learning

- First of all, we need to select GPU in the runtime menu

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


- To check which GPU type you are using

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


##1.2. Install Hugging Face Library

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 7.1MB/s 
     |████████████████████████████████| 890kB 37.7MB/s 
     |████████████████████████████████| 1.1MB 28.5MB/s 
     |████████████████████████████████| 3.0MB 38.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=467bd250729e7e500207ab58e4386842b47c555be7f337545341dc4ec63f3c8d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


#2.Corpus Data

##2.1.Mount Google Drive to this Notebook instance.

In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


- To check the location of this drive

In [ ]:
!ls

drive  sample_data


In [ ]:
!ls drive/My\ Drive/BERT/SM/KoBERT/Postposition/Data

accuracy     test_Eyse.csv  train_Ey.csv    train_Lo.csv
test_Ey.csv  test_Lo.csv    train_Eyse.csv  t-SNE


##2.2.Install required packages

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

##2.3.Importing files to generate data

In [ ]:
fileDir = "drive/My Drive/BERT/SM/KoBERT/Postposition/Data/test_Lo.csv"
fr = open(fileDir, 'r')
contents= fr.readlines()
fr.close()

test = pd.DataFrame(columns=('index', 'Label', 'Sentence'))
i = 0
index = ""
label = ""
sentence = ""
for content in contents:
    if i == 0:
        pass
    else:
        infos = content.split(",")
        index = infos[0]
        label = int(infos[1])
        sentence = infos[2].replace("\n","")
        test.loc[i] = [index, label, sentence]
    i = i + 1

print(test)

    index Label                         Sentence
1       0     0  이 넥타이는 수제품으로 우리나라에서는 하나밖에 없습니다.
2       1     0         모든 주장이 나름대로의 근거를 갖추고 있다.
3       2     0                 멍멍한 채로 시간이 흘러갔다.
4       3     0             만찬 시에는 반주로 마주앙이 나온다.
5       4     0          아내의 얼굴이 절망적으로 굳어지고 있었다.
..    ...   ...                              ...
463   462     5           무대 위로 무수한 꽃송이들이 날아오른다.
464   463     5       오른쪽 차창 밖으로는 끝없이 이어지는 바다였다.
465   464     5          철장 사이로 쳐다봐야 하는 형제들의 아픔!
466   465     5                  출근은 우선 공보부로 했다.
467   466     5              그분이 나를 문예반으로 끌어들였다.

[467 rows x 3 columns]


In [ ]:
fileDir = "drive/My Drive/BERT/SM/KoBERT/Postposition/Data/train_Lo.csv"
fr = open(fileDir, 'r')
contents= fr.readlines()
fr.close()

train = pd.DataFrame(columns=('index', 'Label', 'Sentence'))
i = 0
index = ""
label = ""
sentence = ""
for content in contents:
    if i == 0:
        pass
    else:
        infos = content.split(",")
        index = infos[0]
        label = int(infos[1])
        sentence = infos[2].replace("\n","")
        train.loc[i] = [index, label, sentence]
    i = i + 1

print(train)

     index Label                        Sentence
1        0     0     그러나 진공 상태로 포장되어 있는 사랑이란 없다.
2        1     0             한 마디로 하여 정신병자인 것이다.
3        2     0         복수를 일본어로는 '가따끼우찌'라고 한다.
4        3     0                 마지막으로 "기죽지 마라."
5        4     0  관세음 보살 불교에서 대자대비의 상징으로 받드는 보살.
...    ...   ...                             ...
4237  4236     5                 그다지 크지 않은 창너머로.
4238  4237     5                      펑크난 혼 사이로.
4239  4238     5       마카오 친구 집으로 돌아다님 86.5­87.9
4240  4239     5             양편으로 사람들이 주욱 서 있었다.
4241  4240     5                 밤에 벽으로 다니는 곤충).

[4241 rows x 3 columns]


- 6 functions of Korean postposition '-Lo' <br>
0: FNS (final state)<br>
1: INS (instrumental)<br>
2: DIR (Direction)<br>
3: EFF (effector)<br>
4: CRT (criterion)<br>
5: LOC (location)<br>


In [ ]:
print(train[0:20])
print(train.shape)
print(test.shape)

   index Label                        Sentence
1      0     0     그러나 진공 상태로 포장되어 있는 사랑이란 없다.
2      1     0             한 마디로 하여 정신병자인 것이다.
3      2     0         복수를 일본어로는 '가따끼우찌'라고 한다.
4      3     0                 마지막으로 "기죽지 마라."
5      4     0  관세음 보살 불교에서 대자대비의 상징으로 받드는 보살.
6      5     0               일곱시간을 내리 입석으로 갔다.
7      6     0        그런 개는 천상 씨받이로나 쓰일 뿐이지요."
8      7     0           그 댓가로 돈을 요구한 병사도 있었다.
9      8     0                  조금쯤 슬퍼하는 표정으로.
10     9     0               「마지막으로 한말씀 더 드리죠.
11    10     0              「다시 그 사람 얘기로 돌아가죠.
12    11     0             마치 포장을 하듯 어둠으로 말이야.
13    12     0              우산 생산을 보기로 생각해 보자.
14    13     0                 지리여행으로 알맞은 곳이다.
15    14     0             위험사회는 참여로 탈출할 수 있다.
16    15     0                 조카가 혼잣말로 중얼거렸다.
17    16     0       언제나 저를 신중한 사람으로 믿고 있었거든요.
18    17     0               그때부터 나는 영옥이로 통했다.
19    18     0     정 사장은 공인회계사 출신으로 재무 관리에 밝다.
20    19     0             숨이 막힐 정도로 격렬한 키스였다.
(4241, 3)
(46

##2.4.Cleaning the corpus data; remove the punctuations

In [ ]:
#정제하기

train['Sentence'] = train['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\\n\t]+', " ", regex=True)
test['Sentence'] = test['Sentence'].str.replace(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', " ", regex=True)
train['Sentence'] = train['Sentence'].str.replace(r'\t+', " ", regex=True)
test['Sentence'] = test['Sentence'].str.replace(r'\t+', " ", regex=True)
train['Sentence'] = train['Sentence'].str.replace(r'[\\n]+'," ", regex=True)
test['Sentence'] = test['Sentence'].str.replace(r'[\\n]+'," ", regex=True)

In [ ]:
train.head(5)
test.head(5)

,index,Label,Sentence
1,0,0,이 넥타이는 수제품으로 우리나라에서는 하나밖에 없습니다
2,1,0,모든 주장이 나름대로의 근거를 갖추고 있다
3,2,0,멍멍한 채로 시간이 흘러갔다
4,3,0,만찬 시에는 반주로 마주앙이 나온다
5,4,0,아내의 얼굴이 절망적으로 굳어지고 있었다


#3.Data processing


##3.1 Preprocessing

- Load the Tokenization.py for KoBERT

In [ ]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

- Convert each sentence to BERT format

In [ ]:
# 리뷰 문장 추출
sentences = train['Sentence']
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 그러나 진공 상태로 포장되어 있는 사랑이란 없다  [SEP]',
 '[CLS] 한 마디로 하여 정신병자인 것이다  [SEP]',
 '[CLS] 복수를 일본어로는  가따끼우찌 라고 한다  [SEP]',
 '[CLS] 마지막으로  기죽지 마라  [SEP]',
 '[CLS] 관세음 보살 불교에서 대자대비의 상징으로 받드는 보살  [SEP]',
 '[CLS] 일곱시간을 내리 입석으로 갔다  [SEP]',
 '[CLS] 그런 개는 천상 씨받이로나 쓰일 뿐이지요  [SEP]',
 '[CLS] 그 댓가로 돈을 요구한 병사도 있었다  [SEP]',
 '[CLS] 조금쯤 슬퍼하는 표정으로  [SEP]',
 '[CLS] 「마지막으로 한말씀 더 드리죠  [SEP]']

- Save the label data

In [ ]:
# 라벨 추출
labels = train['Label'].values
labels_re = []
for label in labels:
  labels_re.append(label)
labels = labels_re

- Check the KoBertTokenizer

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.



[CLS] 그러나 진공 상태로 포장되어 있는 사랑이란 없다  [SEP]
['[CLS]', '▁그러나', '▁진', '공', '▁상태', '로', '▁포', '장', '되어', '▁있는', '▁사랑', '이란', '▁없다', '[SEP]']


In [ ]:
 # 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   2, 1199, 4360, 5452, 2680, 6079, 4856, 7178, 5894, 3860, 2590,
       7107, 3273,    3,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])
print(labels)
print(input_ids)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   2, 3647, 7376, 7078,  517, 6819, 7074, 6586,    3,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(2)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# 리뷰 문장 추출
sentences = test['Sentence']
sentences[:10]

1     이 넥타이는 수제품으로 우리나라에서는 하나밖에 없습니다 
2            모든 주장이 나름대로의 근거를 갖추고 있다 
3                    멍멍한 채로 시간이 흘러갔다 
4                만찬 시에는 반주로 마주앙이 나온다 
5             아내의 얼굴이 절망적으로 굳어지고 있었다 
6           그러나 어머니의 교회행은 그것으로 끝이 났다 
7                  그 사실이 진정으로 나는 기뻤다 
8                 이런 녀석은 진짜로 혼이 나야 해 
9         둘이 말없이 그런 상태로 한참 동안 앉아 있었다 
10               처음으로 바깥 풍경이 눈에 들어왔다 
Name: Sentence, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 이 넥타이는 수제품으로 우리나라에서는 하나밖에 없습니다  [SEP]',
 '[CLS] 모든 주장이 나름대로의 근거를 갖추고 있다  [SEP]',
 '[CLS] 멍멍한 채로 시간이 흘러갔다  [SEP]',
 '[CLS] 만찬 시에는 반주로 마주앙이 나온다  [SEP]',
 '[CLS] 아내의 얼굴이 절망적으로 굳어지고 있었다  [SEP]',
 '[CLS] 그러나 어머니의 교회행은 그것으로 끝이 났다  [SEP]',
 '[CLS] 그 사실이 진정으로 나는 기뻤다  [SEP]',
 '[CLS] 이런 녀석은 진짜로 혼이 나야 해  [SEP]',
 '[CLS] 둘이 말없이 그런 상태로 한참 동안 앉아 있었다  [SEP]',
 '[CLS] 처음으로 바깥 풍경이 눈에 들어왔다  [SEP]']

In [ ]:
# 라벨 추출
labels = test['Label'].values
labels_re = []
for label in labels:
  labels_re.append(label)
labels = labels_re

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


[CLS] 이 넥타이는 수제품으로 우리나라에서는 하나밖에 없습니다  [SEP]
['[CLS]', '▁이', '▁', '넥', '타', '이', '는', '▁수', '제품', '으로', '▁우리나라', '에서는', '▁하나', '밖에', '▁없', '습니다', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   2, 3647,  517, 5705, 7581, 7096, 5760, 2872, 7239, 7078, 3503,
       6904, 4928, 6285, 3270, 6701,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   2, 3647,  517, 5705, 7581, 7096, 5760, 2872, 7239, 7078, 3503, 6904,
        4928, 6285, 3270, 6701,    3,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
test_dataloader

#4.Create model

##4.1. Load the KoBERT model; BertForSequenceClassification

In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=6)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

##4.2. Create the functions for checking accuracy rate

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
def FNS_flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    match_num = 0
    func_num = 0
    for i in range(0,len(pred_flat)):
      if (pred_flat[i] == labels_flat[i]) and (labels_flat[i] == 0):
        match_num += 1
      if labels_flat[i] == 0:
        func_num += 1

    if match_num == 0 or func_num == 0:
      return 0
    else:
      return match_num / func_num

def INS_flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    match_num = 0
    func_num = 0
    for i in range(0,len(pred_flat)):
      if (pred_flat[i] == labels_flat[i]) and (labels_flat[i] == 1):
        match_num += 1
      if labels_flat[i] == 1:
        func_num += 1

    if match_num == 0 or func_num == 0:
      return 0
    else:
      return match_num / func_num

def DIR_flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    match_num = 0
    func_num = 0
    for i in range(0,len(pred_flat)):
      if (pred_flat[i] == labels_flat[i]) and (labels_flat[i] == 2):
        match_num += 1
      if labels_flat[i] == 2:
        func_num += 1

    if match_num == 0 or func_num == 0:
      return 0
    else:
      return match_num / func_num

def EFF_flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    match_num = 0
    func_num = 0
    for i in range(0,len(pred_flat)):
      if (pred_flat[i] == labels_flat[i]) and (labels_flat[i] == 3):
        match_num += 1
      if labels_flat[i] == 3:
        func_num += 1

    if match_num == 0 or func_num == 0:
      return 0
    else:
      return match_num / func_num

def CRT_flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    match_num = 0
    func_num = 0
    for i in range(0,len(pred_flat)):
      if (pred_flat[i] == labels_flat[i]) and (labels_flat[i] == 4):
        match_num += 1
      if labels_flat[i] == 4:
        func_num += 1

    if match_num == 0 or func_num == 0:
      return 0
    else:
      return match_num / func_num

def LOC_flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    match_num = 0
    func_num = 0
    for i in range(0,len(pred_flat)):
      if (pred_flat[i] == labels_flat[i]) and (labels_flat[i] == 5):
        match_num += 1
      if labels_flat[i] == 5:
        func_num += 1

    if match_num == 0 or func_num == 0:
      return 0
    else:
      return match_num / func_num

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

##4.3. Train the corpus data with KoBERT

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 50

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

final_info = {}

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    FNS_nb_eval_steps, FNS_eval_accuracy = 0, 0
    INS_nb_eval_steps, INS_eval_accuracy = 0, 0
    DIR_nb_eval_steps, DIR_eval_accuracy = 0, 0
    EFF_nb_eval_steps, EFF_eval_accuracy = 0, 0
    CRT_nb_eval_steps, CRT_eval_accuracy = 0, 0
    LOC_nb_eval_steps, LOC_eval_accuracy = 0, 0

    epoch_info = {}

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in test_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

        FNS_tmp_eval_accuracy = FNS_flat_accuracy(logits, label_ids)
        FNS_eval_accuracy += FNS_tmp_eval_accuracy
        FNS_nb_eval_steps += 1

        INS_tmp_eval_accuracy = INS_flat_accuracy(logits, label_ids)
        INS_eval_accuracy += INS_tmp_eval_accuracy
        INS_nb_eval_steps += 1

        DIR_tmp_eval_accuracy = DIR_flat_accuracy(logits, label_ids)
        DIR_eval_accuracy += DIR_tmp_eval_accuracy
        DIR_nb_eval_steps += 1

        EFF_tmp_eval_accuracy = EFF_flat_accuracy(logits, label_ids)
        EFF_eval_accuracy += EFF_tmp_eval_accuracy
        EFF_nb_eval_steps += 1

        CRT_tmp_eval_accuracy = CRT_flat_accuracy(logits, label_ids)
        CRT_eval_accuracy += CRT_tmp_eval_accuracy
        CRT_nb_eval_steps += 1

        LOC_tmp_eval_accuracy = LOC_flat_accuracy(logits, label_ids)
        LOC_eval_accuracy += LOC_tmp_eval_accuracy
        LOC_nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("  Detail accuracy  ")
    print("  FNS_Accuracy: {0:.2f}".format(FNS_eval_accuracy/FNS_nb_eval_steps))
    print("  INS_Accuracy: {0:.2f}".format(INS_eval_accuracy/INS_nb_eval_steps))
    print("  DIR_Accuracy: {0:.2f}".format(DIR_eval_accuracy/DIR_nb_eval_steps))
    print("  EFF_Accuracy: {0:.2f}".format(EFF_eval_accuracy/EFF_nb_eval_steps))
    print("  CRT_Accuracy: {0:.2f}".format(CRT_eval_accuracy/CRT_nb_eval_steps))
    print("  LOC_Accuracy: {0:.2f}".format(LOC_eval_accuracy/LOC_nb_eval_steps))

    epoch_info["Total"] = round(eval_accuracy/nb_eval_steps,3)
    epoch_info["Loss"] = round(avg_train_loss,3)
    epoch_info["FNS"] = round(FNS_eval_accuracy/FNS_nb_eval_steps,3)
    epoch_info["INS"] = round(INS_eval_accuracy/INS_nb_eval_steps,3)
    epoch_info["DIR"] = round(DIR_eval_accuracy/DIR_nb_eval_steps,3)
    epoch_info["EFF"] = round(EFF_eval_accuracy/EFF_nb_eval_steps,3)
    epoch_info["CRT"] = round(CRT_eval_accuracy/CRT_nb_eval_steps,3)
    epoch_info["LOC"] = round(LOC_eval_accuracy/LOC_nb_eval_steps,3)

    final_info["epoch"+str(epoch_i)] = epoch_info


    # 평가모드로 변경
    model.eval()
    test_input_ids = []
    test_input_mask = []
    test_labels = []

    num = 0
    for step, batch in enumerate(test_data):   #467, 128
      # print("batch",batch)
      # 배치를 GPU에 넣음
      batch = tuple(t.to(device) for t in batch)
      
      # 배치에서 데이터 추출
      b_input_ids, b_input_mask, b_labels = batch
      input_ids_arr = []
      input_mask_arr = []

      

      for i in range(0,len(b_input_ids)):
        input_ids_arr.append(int(b_input_ids[i]))
        input_mask_arr.append(int(b_input_mask[i]))

      
      test_input_ids.append(input_ids_arr)
      test_input_mask.append(input_mask_arr)
      test_labels.append(int(b_labels))


    test_input_ids = torch.tensor(test_input_ids)
    test_input_mask = torch.tensor(test_input_mask)
    test_labels = test_labels

    test_input_ids = test_input_ids.to(device)
    test_input_mask = test_input_mask.to(device)


    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(test_input_ids, 
                        token_type_ids=None, 
                        attention_mask=test_input_mask)
        

    sentence_vecs_sum = outputs[0]

    sentence_array = []
    for i in range(0,len(sentence_vecs_sum)):
      each_array = []
      for j in range(0,len(sentence_vecs_sum[i])):
        each_array.append(float(sentence_vecs_sum[i][j]))
      sentence_array.append(each_array)

    initial_df = pd.DataFrame(sentence_array)

    from sklearn.manifold import TSNE
    tsne = TSNE(n_components=2, random_state=0)
    tsne_obj= tsne.fit_transform(initial_df)

    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],'Y':tsne_obj[:,1],'Label':test_labels})

    # tsne_df.to_csv("drive/My Drive/BERT/SM/KoBERT/Postposition/Data/t-SNE/Lo_tSNE_epoch_"+str(epoch_i)+".csv")

    import numpy as np   
    import pandas as pd 
    from plotnine import *

    print("")
    print("  Network visualization  ")
    print(ggplot(tsne_df, aes(x='X', y='Y')) + geom_point(aes(colour = 'Label')))

print("")
print("Training complete!")
print("")
print("Final result is below!")
print(final_info)

# f = open("drive/My Drive/BERT/SM/KoBERT/Postposition/Data/accuracy/Lo_accuracy_epoch_"+str(epochs)+".txt", 'w')
# f.write(str(final_info))
# f.close()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
model.save_pretrained('drive/My Drive/BERT/SM/KoBERT/Postposition/Model/')
tokenizer.save_pretrained('drive/My Drive/BERT/SM/KoBERT/Postposition/Model/')

('drive/My Drive/BERT/SM/KoBERT/Postposition/Model/tokenizer_78b3253a26.model',
 'drive/My Drive/BERT/SM/KoBERT/Postposition/Model/vocab.txt',
 'drive/My Drive/BERT/SM/KoBERT/Postposition/Model/special_tokens_map.json',
 'drive/My Drive/BERT/SM/KoBERT/Postposition/Model/added_tokens.json')

- additional test with test_dataloader

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):  #15
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    # 로스 구함
    logits = outputs[0]  #32 , 8

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.85
Test took: 0:00:07


#5.Create function using trained model

##5.1. Create functions

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

##5.2. Check the result with real sentence

In [ ]:
logits = test_sentences(['나는 집으로 가고 있습니다.'])

print(logits)
print(np.argmax(logits))

[[-2.5875225 -1.795376   9.1881895 -2.2699857 -2.0289989 -1.7857274]]
2


In [ ]:
logits = test_sentences(['나는 가위로 종이를 잘랐습니다.'])

print(logits)
print(np.argmax(logits))

[[-1.6707162  8.955514  -1.5057735 -1.4657849 -1.4273801 -1.6496873]]
1


#6.Conclusion

##6.1. Summary
- 

##6.2. Suggestion
- 